## Import libraries

In [1]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/observatorio/Documents/repos/accesibilidad-urbana


In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

# Correlation calc
import scipy.stats as stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Notebook config

In [3]:
city = 'Guadalajara'
res = '11'

## Input

In [4]:
# Hexs with temperature data (Have no mun data)
temperature_gdf = gpd.read_file(str(project_root)+f"/data/external/temporal_fromjupyter/temp_check/temperature_res{res}_{city}.gpkg")

# Show
print(temperature_gdf.shape)
temperature_gdf.head(2)

(271809, 19)


,hex_id,res,temperature_mean,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,temperature_2018,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city,temperature_anomaly,temperature_slope,geometry
0,8b49ab592d69fff,11,38.983696,7.361305,39.214286,48.684524,28.000000,20.684524,0.020833,36.984694,40.078571,39.607143,37.285714,39.875000,39.266234,Guadalajara,1.604601,0.242159,"POLYGON ((668606.368 2281504.51, 668624.258 22..."
1,8b498c96c79bfff,11,37.422554,7.359019,37.640625,46.869792,26.765625,20.104167,0.024148,35.468750,38.456250,38.156250,35.518750,37.896875,38.096591,Guadalajara,0.043460,0.252102,"POLYGON ((669547.996 2285989.466, 669565.887 2..."


In [6]:
# Hexs from database (have mun data)
hexs_schema = 'hexgrid'
hexs_table = f'hexgrid_{res}_city_2020'
query = f"SELECT * FROM {hexs_schema}.{hexs_table} WHERE \"city\" = '{city}\'"
hexs_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(hexs_gdf.shape)
hexs_gdf.head(2)

(976710, 6)


,hex_id_11,geometry,CVEGEO,NOMGEO,city,type
0,8b49aa2d9936fff,"POLYGON ((-103.32293 20.57961, -103.32275 20.5...",14070,El Salto,Guadalajara,urban
1,8b498c960081fff,"POLYGON ((-103.33538 20.63189, -103.33521 20.6...",14039,Guadalajara,Guadalajara,urban


## Merge

In [9]:
temp_mun_gdf = pd.merge(temperature_gdf,hexs_gdf[['hex_id_11','NOMGEO']], left_on='hex_id', right_on='hex_id_11')
temp_mun_gdf.drop(columns=['hex_id_11'],inplace=True)
temp_mun_gdf.rename(columns={'NOMGEO':'nomgeo'},inplace=True)

# Show
print(temp_mun_gdf.shape)
temp_mun_gdf.head(2)

(271809, 20)


,hex_id,res,temperature_mean,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,temperature_2018,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city,temperature_anomaly,temperature_slope,geometry,nomgeo
0,8b49ab592d69fff,11,38.983696,7.361305,39.214286,48.684524,28.000000,20.684524,0.020833,36.984694,40.078571,39.607143,37.285714,39.875000,39.266234,Guadalajara,1.604601,0.242159,"POLYGON ((668606.368 2281504.51, 668624.258 22...",Guadalajara
1,8b498c96c79bfff,11,37.422554,7.359019,37.640625,46.869792,26.765625,20.104167,0.024148,35.468750,38.456250,38.156250,35.518750,37.896875,38.096591,Guadalajara,0.043460,0.252102,"POLYGON ((669547.996 2285989.466, 669565.887 2...",Guadalajara


## Analyse by municipality

In [11]:
grouped_by = temp_mun_gdf.groupby('nomgeo').agg({'temperature_anomaly':np.mean,
                                                 'temperature_slope':np.mean})

# Show
grouped_by

,temperature_anomaly,temperature_slope
nomgeo,,
El Salto,0.576784,0.324694
Guadalajara,0.528679,0.318289
Juanacatlán,-0.071641,0.374280
San Pedro Tlaquepaque,0.417185,0.343766
Tlajomulco de Zúñiga,0.175391,0.144324
Tonalá,0.081861,0.298470
Zapopan,-0.722021,0.242723
